<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-experiment/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 32.6 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
11.8


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu118
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone -b mmajid/tgnn-improvements https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
import util as util
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 116 (delta 57), reused 49 (delta 14), pack-reused 0
Receiving objects: 100% (116/116), 91.76 KiB | 824.00 KiB/s, done.
Resolving deltas: 100% (57/57), done.


In [5]:
!pip install -U torch-geometric-temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.0 MB/s eta 0:00:00
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=2ed608208e61fd9daf2a89c918dc2c4825a78ac08414cbb7562725c0cf263ce9
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
Successfully built torch-geometric-temporal
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires pyth

In [6]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



# # look for other datasets
from torch_geometric_temporal.dataset import METRLADatasetLoader
loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
iterator = iter(dataset)
print("METRLADatasetLoader", next(iterator))
print("METRLADatasetLoader", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset()
iterator = iter(dataset)
print("ChickenpoxDatasetLoader", next(iterator))
print("ChickenpoxDatasetLoader", next(iterator))

# from torch_geometric_temporal.dataset import PedalMeDatasetLoader
# dataset = PedalMeDatasetLoader().get_dataset()
# print("PedalMeDatasetLoader", next(iter(dataset)))


# from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
# dataset = WikiMathsDatasetLoader().get_dataset(lags=14)
# iterator = iter(dataset)
# print("WikiMathsDatasetLoader", next(iterator))
# print("WikiMathsDatasetLoader", next(iterator))

# from torch_geometric_temporal.dataset import WindmillOutputSmallDatasetLoader
# dataset = WindmillOutputSmallDatasetLoader().get_dataset()
# print("WindmillOutputSmallDatasetLoader", next(iter(dataset)))

# from torch_geometric_temporal.dataset import EnglandCovidDatasetLoader
# dataset = EnglandCovidDatasetLoader().get_dataset()
# print("EnglandCovidDatasetLoader", next(iter(dataset)))


# from torch_geometric_temporal.dataset import MontevideoBusDatasetLoader
# dataset = MontevideoBusDatasetLoader().get_dataset()
# print("MontevideoBusDatasetLoader", next(iter(dataset)))


# from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
# dataset = TwitterTennisDatasetLoader().get_dataset()
# print("TwitterTennisDatasetLoader", next(iter(dataset)))

METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])
WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])


In [7]:
# Run WikiMaths Web Traffic Dataset

from torch_geometric_temporal.signal import temporal_signal_split

# sample training loop
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)


import torch
import torch.nn.functional as F

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() # clear cuda cache

#enable extensions
util.extensions_enabled = True
print("Graph Wavenet Extension for Performance Improvements: {}".format(util.extensions_enabled))
in_dim = 4
out_dim = 1
num_nodes = 20
timesteps_to_predict = 1
epochs = 50
lrate = 0.0005
wdecay = 0.0001
save_path = "store/checkpoint"

model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("start training...", flush=True)

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)
scheduler = None
if util.extensions_enabled:
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.03)
clip = 5
if util.extensions_enabled:
    clip = 3

model.train()

for epoch in tqdm(range(epochs)):
    train_loss = []
    train_mape = []
    train_rmse = []
    t1 = time.time()

    for iter, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        y_train = snapshot.y.to(device)
        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mae(pred, y_train, 0.0) # mean absolute error as loss
        loss.backward()

        if util.extensions_enabled:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        if util.extensions_enabled:
            scheduler.step()
        optimizer.zero_grad()

        mape = masked_mape(pred, y_train, 0.0).item()
        rmse = masked_rmse(pred, y_train, 0.0).item()
        metrics = loss.item(), mape, rmse
        train_loss.append(metrics[0])
        train_mape.append(metrics[1])
        train_rmse.append(metrics[2])

        if iter % 100 == 0:
            log = 'Iter: {:05d}, Train Loss: {:.4f}, Train MAPE: ' + \
                '{:.4f}, Train RMSE: {:.4f}'
            print(
                log.format(iter, train_loss[-1], train_mape[-1],
                            train_rmse[-1]), flush=True)



# eval
model.eval()
loss = 0
for iter, snapshot in enumerate(test_dataset):
    x_test = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
    y_test = snapshot.y.to(device)

    with torch.no_grad():
        pred = model(x_test, snapshot.edge_index, snapshot.edge_attr)
    loss += masked_mae(pred, y_test, 0.0) # mean absolute error as loss

loss = loss / (iter+1)
loss = loss.item()
print("MAE: {:.4f}".format(loss))


start training...


  0%|          | 0/50 [00:00<?, ?it/s]

Iter: 00000, Train Loss: 0.8241, Train MAPE: -0.9656, Train RMSE: 0.9297
Iter: 00100, Train Loss: 0.3730, Train MAPE: -0.2377, Train RMSE: 0.6042
Iter: 00200, Train Loss: 0.4254, Train MAPE: 0.3083, Train RMSE: 0.7137
Iter: 00300, Train Loss: 0.3780, Train MAPE: 1.3412, Train RMSE: 0.6633
Iter: 00400, Train Loss: 0.3458, Train MAPE: -0.4844, Train RMSE: 0.5490
Iter: 00500, Train Loss: 0.3241, Train MAPE: 4.1139, Train RMSE: 0.4597


  2%|▏         | 1/50 [00:42<35:05, 42.96s/it]

Iter: 00000, Train Loss: 0.6678, Train MAPE: -0.8914, Train RMSE: 0.7701
Iter: 00100, Train Loss: 0.3278, Train MAPE: 0.6174, Train RMSE: 0.5522
Iter: 00200, Train Loss: 0.4119, Train MAPE: 0.3628, Train RMSE: 0.7039
Iter: 00300, Train Loss: 0.3689, Train MAPE: 2.2443, Train RMSE: 0.6732
Iter: 00400, Train Loss: 0.3234, Train MAPE: -0.1467, Train RMSE: 0.5359
Iter: 00500, Train Loss: 0.3117, Train MAPE: 4.1014, Train RMSE: 0.4499


...

Iter: 00000, Train Loss: 0.2840, Train MAPE: -0.1978, Train RMSE: 0.4166
Iter: 00100, Train Loss: 0.3217, Train MAPE: 0.4770, Train RMSE: 0.5371
Iter: 00200, Train Loss: 0.4027, Train MAPE: 0.4830, Train RMSE: 0.6838
Iter: 00300, Train Loss: 0.3623, Train MAPE: 1.2332, Train RMSE: 0.6460
Iter: 00400, Train Loss: 0.3063, Train MAPE: 0.7465, Train RMSE: 0.5236
Iter: 00500, Train Loss: 0.2794, Train MAPE: 4.6911, Train RMSE: 0.4365


 98%|█████████▊| 49/50 [27:10<00:33, 33.21s/it]

Iter: 00000, Train Loss: 0.2877, Train MAPE: -0.2040, Train RMSE: 0.4199
Iter: 00100, Train Loss: 0.3325, Train MAPE: -0.0128, Train RMSE: 0.5476
Iter: 00200, Train Loss: 0.4058, Train MAPE: 0.5338, Train RMSE: 0.6862
Iter: 00300, Train Loss: 0.3139, Train MAPE: 0.9375, Train RMSE: 0.6294
Iter: 00400, Train Loss: 0.3139, Train MAPE: 0.0992, Train RMSE: 0.5229
Iter: 00500, Train Loss: 0.2798, Train MAPE: 4.3636, Train RMSE: 0.4352


100%|██████████| 50/50 [27:42<00:00, 33.26s/it]


MAE: 0.7212


In [8]:
!nvidia-smi

Mon Dec 11 10:50:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    40W /  70W |   1711MiB / 15360MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
# Reset GPU RAM if GPU out of memory

# !pip install numba

# from numba import cuda

# cuda.select_device(0) # choosing second GPU

# cuda.close()